<a href="https://colab.research.google.com/github/shahtvisha/TextSummarizationUsingDeepLearning/blob/main/TextSummarizationEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=27f482862e0861b9a5c19483d9849bf0db6fb4eb0f1a08230ce9dca8c92950de
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [ ]:
pip install datasets==2.21.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.0
    Uninstalling datasets-3.0.0:
      Successfully uninstalled datasets-3.0.0


In [ ]:
from datasets import load_metric
import torch
import math

# Load ROUGE metric
rouge = load_metric("rouge")


In [ ]:
import pandas as pd
csv_file_path = "/content/preprocessed_pubmed_dataset(1).xls"
pubmed_data = pd.read_csv(csv_file_path)
pubmed_data['article'] = pubmed_data['article'].astype(str)
pubmed_data['abstract'] = pubmed_data['abstract'].astype(str)
pubmed_data.dropna()
pubmed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   article   473 non-null    object
 1   abstract  473 non-null    object
dtypes: object(2)
memory usage: 7.5+ KB


In [ ]:
pubmed_data['article'] = pubmed_data['article'].apply(lambda x: str(x))
pubmed_data['abstract'] = pubmed_data['abstract'].apply(lambda x: str(x))
pubmed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   article   473 non-null    object
 1   abstract  473 non-null    object
dtypes: object(2)
memory usage: 7.5+ KB


In [ ]:
pubmed_data['article'].str.len().mean()

16547.44820295983

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 843.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
from transformers import (
    PegasusTokenizer, PegasusForConditionalGeneration,
    T5Tokenizer, T5ForConditionalGeneration,
    GPT2Tokenizer, GPT2LMHeadModel
)

from datasets import load_metric
import nltk
import sacrebleu
import pandas as pd

nltk.download('punkt')

# Define variables
max_iterations = 50  # Limit iterations if needed
chunk_size = 512  # Set based on your memory constraints for each model

# Load evaluation metrics
rouge = load_metric("rouge")

# Function to compute BLEU score using sacrebleu
def compute_bleu(preds, labels):
    return sacrebleu.corpus_bleu(preds, [labels]).score

# Function to evaluate a generated summary
def evaluate_summary(generated_summary, reference):
    # Compute ROUGE scores
    rouge_scores = rouge.compute(predictions=[generated_summary], references=[reference])

    # Compute BLEU score
    bleu_score = compute_bleu(generated_summary, reference)

    return rouge_scores, {"bleu": bleu_score}

# Function to generate summary
def generate_summary(model, tokenizer, text, chunk_size):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=chunk_size)
    summary_ids = model.generate(inputs.input_ids)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
results = []

# List of models to evaluate
models = [
    ('Pegasus', PegasusTokenizer.from_pretrained('google/pegasus-xsum'), PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')),
    ('T5', T5Tokenizer.from_pretrained('t5-small'), T5ForConditionalGeneration.from_pretrained('t5-small')),
    ('BART', BartTokenizer.from_pretrained('facebook/bart-large-cnn'), BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn'))]

for i, (model_name, tokenizer, model) in enumerate(models):
    if model_name == 'GPT':
        tokenizer.pad_token = tokenizer.eos_token  # Set the eos_token as pad_token for GPT

# Iterate over each model
for model_name, tokenizer, model in models:
    print(f"Evaluating {model_name}...")
    model_results = {
        "model": model_name,
        "rouge1": [],
        "rouge2": [],
        "rougeL": [],
        "bleu": []
    }

    # Iterate over PubMed dataset
    for i in range(min(max_iterations, len(pubmed_data))):
        row = pubmed_data.iloc[i]
        article = row['article']
        reference = row['abstract']

        # Generate summary
        generated_summary = generate_summary(model, tokenizer, article, chunk_size)

        # Evaluate
        rouge_scores, bleu_scores = evaluate_summary(generated_summary, reference)

        # Save metrics
        print(f"ROUGE Scores for {model_name}: {rouge_scores}")
        print(f"BLEU Scores for {model_name}: {bleu_scores}")

        model_results['rouge1'].append(rouge_scores['rouge1'].mid.fmeasure)
        model_results['rouge2'].append(rouge_scores['rouge2'].mid.fmeasure)
        model_results['rougeL'].append(rouge_scores['rougeL'].mid.fmeasure)
        model_results['bleu'].append(bleu_scores['bleu'])

        print(f"Processed {i + 1}/{max_iterations}")

    # Calculate average metrics for the model
    avg_rouge1 = sum(model_results['rouge1']) / len(model_results['rouge1'])
    avg_rouge2 = sum(model_results['rouge2']) / len(model_results['rouge2'])
    avg_rougeL = sum(model_results['rougeL']) / len(model_results['rougeL'])
    avg_bleu = sum(model_results['bleu']) / len(model_results['bleu'])

    # Append final averages to results list
    results.append([model_name, avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu])

# Convert results to a DataFrame for easy tabular format
results_df = pd.DataFrame(results, columns=['Model', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU'])

# Display results in tabular format
print(results_df)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Evaluating Pegasus...
ROUGE Scores for Pegasus: {'rouge1': AggregateScore(low=Score(precision=0.6451612903225806, recall=0.08733624454148471, fmeasure=0.15384615384615385), mid=Score(precision=0.6451612903225806, recall=0.08733624454148471, fmeasure=0.15384615384615385), high=Score(precision=0.6451612903225806, recall=0.08733624454148471, fmeasure=0.15384615384615385)), 'rouge2': AggregateScore(low=Score(precision=0.26666666666666666, recall=0.03508771929824561, fmeasure=0.06201550387596899), mid=Score(precision=0.26666666666666666, recall=0.03508771929824561, fmeasure=0.06201550387596899), high=Score(precision=0.26666666666666666, recall=0.03508771929824561, fmeasure=0.06201550387596899)), 'rougeL': AggregateScore(low=Score(precision=0.45161290322580644, recall=0.0611353711790393, fmeasure=0.10769230769230768), mid=Score(precision=0.45161290322580644, recall=0.0611353711790393, fmeasure=0.10769230769230768), high=Score(precision=0.45161290322580644, recall=0.0611353711790393, fmeasure

T5: 0.05007747843014379,
0.015463318968218004,
0.042152889712906816,
0.0,

Pegasus:

0.14930152670229735,
0.045761377876343434,
0.10970789942084312,
0.0,

Tabular:  Model   ROUGE-1   ROUGE-2   ROUGE-L  BLEU
0  Pegasus  0.149302  0.045761  0.109708   0.0
1       T5  0.050077  0.015463  0.042153   0.0
2     BART  0.215820  0.069008  0.142576   0.0

In [ ]:
print(results_df)

     Model   ROUGE-1   ROUGE-2   ROUGE-L  BLEU
0  Pegasus  0.149302  0.045761  0.109708   0.0
1       T5  0.050077  0.015463  0.042153   0.0
2     BART  0.215820  0.069008  0.142576   0.0


BIOT5

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "TvishaShah/BioT5-layer"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
def summarize(text, model, tokenizer, max_input_length=512, max_output_length=150):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_output_length, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
batch_size = 1
max_iterations = 50

# Prepare to calculate ROUGE scores
rouge = load_metric("rouge")
rouge_scores_list = []

# Iterate over the dataset with a limit of 50 iterations
for i in range(max_iterations):
    row = pubmed_data.iloc[i]
    generated_summary = summarize(row['article'], model, tokenizer)
    rouge_scores = rouge.compute(predictions=[generated_summary], references=[row['abstract']])
    rouge_scores_list.append(rouge_scores)
    print(f"Processed {i + 1}/{max_iterations}")

# Calculate average ROUGE scores over all batches
average_rouge_scores = {
    "rouge1": sum(score["rouge1"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list),
    "rouge2": sum(score["rouge2"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list),
    "rougeL": sum(score["rougeL"].mid.fmeasure for score in rouge_scores_list) / len(rouge_scores_list)
}

Processed 1/50
Processed 2/50
Processed 3/50
Processed 4/50
Processed 5/50
Processed 6/50
Processed 7/50
Processed 8/50
Processed 9/50
Processed 10/50
Processed 11/50
Processed 12/50
Processed 13/50
Processed 14/50
Processed 15/50
Processed 16/50
Processed 17/50
Processed 18/50
Processed 19/50
Processed 20/50
Processed 21/50
Processed 22/50
Processed 23/50
Processed 24/50
Processed 25/50
Processed 26/50
Processed 27/50
Processed 28/50
Processed 29/50
Processed 30/50
Processed 31/50
Processed 32/50
Processed 33/50
Processed 34/50
Processed 35/50
Processed 36/50
Processed 37/50
Processed 38/50
Processed 39/50
Processed 40/50
Processed 41/50
Processed 42/50
Processed 43/50
Processed 44/50
Processed 45/50
Processed 46/50
Processed 47/50
Processed 48/50
Processed 49/50
Processed 50/50


In [ ]:
print(f"Average ROUGE Scores after {max_iterations} iterations:")
print(f"ROUGE-1: {average_rouge_scores['rouge1']:.4f}")
print(f"ROUGE-2: {average_rouge_scores['rouge2']:.4f}")
print(f"ROUGE-L: {average_rouge_scores['rougeL']:.4f}")

Average ROUGE Scores after 50 iterations:
ROUGE-1: 0.2326
ROUGE-2: 0.0780
ROUGE-L: 0.1547


In [ ]:
new_row_df = pd.DataFrame([{
    'Model': 'BioT5-layer',
    'ROUGE-1': average_rouge_scores['rouge1'],
    'ROUGE-2': average_rouge_scores['rouge2'],
    'ROUGE-L': average_rouge_scores['rougeL'],
    'BLEU': None
}])

# Concatenate the new row with the existing DataFrame
results_df = pd.concat([results_df, new_row_df], ignore_index=True)
results_df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU']] = results_df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BLEU']].apply(lambda x: x * 100)
# Display the updated DataFrame
print(results_df)

         Model      ROUGE-1     ROUGE-2      ROUGE-L  BLEU
0      Pegasus  1493.015267  457.613779  1097.078994   0.0
1           T5   500.774784  154.633190   421.528897   0.0
2         BART  2158.198806  690.076671  1425.755728   0.0
3  BioT5-layer  2326.341634  779.750678  1547.465594   NaN
4  BioT5-layer  2326.341634  779.750678  1547.465594   NaN
5  BioT5-layer    23.263416    7.797507    15.474656   NaN


<ipython-input-35-b6d44056cf10>:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row_df], ignore_index=True)


GPT-2

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_model.config.pad_token_id = gpt2_tokenizer.eos_token_id

rouge = load_metric('rouge')
bleu = load_metric('bleu')

def generate_gpt2_summary(model, tokenizer, text, max_new_tokens=150):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def evaluate_summary(generated_summary, reference):
    rouge_scores = rouge.compute(predictions=[generated_summary], references=[reference])

    generated_tokens = gpt2_tokenizer.tokenize(generated_summary)
    reference_tokens = [gpt2_tokenizer.tokenize(reference)]
    bleu_scores = bleu.compute(predictions=[generated_tokens], references=[reference_tokens])

    return rouge_scores, bleu_scores

results = {
    "rouge1": [],
    "rouge2": [],
    "rougeL": [],
    "bleu": []
}

max_iterations = 50

for i in range(min(max_iterations, len(pubmed_data))):
    row = pubmed_data.iloc[i]
    article = row['article']
    reference = row['abstract']

    generated_summary = generate_gpt2_summary(gpt2_model, gpt2_tokenizer, article)

    rouge_scores, bleu_scores = evaluate_summary(generated_summary, reference)

    print(f"ROUGE Scores: {rouge_scores}")
    print(f"BLEU Scores: {bleu_scores}")

    results['rouge1'].append(rouge_scores['rouge1'].mid.fmeasure)
    results['rouge2'].append(rouge_scores['rouge2'].mid.fmeasure)
    results['rougeL'].append(rouge_scores['rougeL'].mid.fmeasure)
    results['bleu'].append(bleu_scores['bleu'])

    print(f"Processed {i + 1}/{max_iterations}")

avg_rouge1 = sum(results['rouge1']) / len(results['rouge1'])
avg_rouge2 = sum(results['rouge2']) / len(results['rouge2'])
avg_rougeL = sum(results['rougeL']) / len(results['rougeL'])
avg_bleu = sum(results['bleu']) / len(results['bleu'])

# Display final average scores for GPT-2
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")
print(f"Average BLEU: {avg_bleu}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ROUGE Scores: {'rouge1': AggregateScore(low=Score(precision=0.1781045751633987, recall=0.4759825327510917, fmeasure=0.25921521997621877), mid=Score(precision=0.1781045751633987, recall=0.4759825327510917, fmeasure=0.25921521997621877), high=Score(precision=0.1781045751633987, recall=0.4759825327510917, fmeasure=0.25921521997621877)), 'rouge2': AggregateScore(low=Score(precision=0.03764320785597381, recall=0.10087719298245613, fmeasure=0.05482717520858164), mid=Score(precision=0.03764320785597381, recall=0.10087719298245613, fmeasure=0.05482717520858164), high=Score(precision=0.03764320785597381, recall=0.10087719298245613, fmeasure=0.05482717520858164)), 'rougeL': AggregateScore(low=Score(precision=0.08660130718954248, recall=0.2314410480349345, fmeasure=0.12604042806183116), mid=Score(precision=0.08660130718954248, recall=0.2314410480349345, fmeasure=0.12604042806183116), high=Score(precision=0.08660130718954248, recall=0.2314410480349345, fmeasure=0.12604042806183116)), 'rougeLsum': 

GPT 2:
Average ROUGE-1: 0.28520048638891604
Average ROUGE-2: 0.11931208539951726
Average ROUGE-L: 0.16678775766947404
Average BLEU: 0.08471726787475778